In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
pd.read_csv("magic04.data")

In [ ]:
cols = ["fLength","fWidth","fSize","fConc","fConc1","fAsym","fM3Long","fM3Trans","fAlpha","fDist","class"]
df = pd.read_csv("magic04.data",names=cols)
df.head()

 here from "flength" to "class" called feature, feature are just thing that we are going to pass our model in order to help us predict the label,which in this case class column.

 here sample 0 has 10 different feature,so i have 10 different values that i can pass into some model.

In [ ]:
df["class"].unique()      #g-gamma h-hydra

In [ ]:
df["class"]=(df['class']=='g').astype(int)

In [ ]:
df.head()

In [ ]:
df

In [ ]:
for labels in cols[:-1]:
  plt.hist(df[df["class"]==1][labels],color='blue',label='gamma',alpha=0.7,density=True)
  plt.hist(df[df["class"]==0][labels],color='red',label='hydron',alpha=0.7,density=True)
  plt.title(labels)
  plt.ylabel("Probability")
  plt.xlabel(labels)
  plt.legend()
  plt.show()

# Train Validate Test

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])      # splitting the dataframe, (df ...) this will shafapal my data, 0.6 means 60% data to validation, 0.8  means 80% data to test

if u seen in df.head() flength has over 100 but in fconc we have less than 100 ; the scale of numbers is way off and somethimes it will effect our results. so we need to scale this relative to mean and deviation of that specific column.

In [ ]:
def scale_dataset(dataframe):
  X = dataframe[dataframe.cols[:-1]].values
  y = dataframe[dataframe.cols[-1]].values

  # import standardScalar

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  # now we'll create the lot data as 1 2d array to do that we need hstack
  # we are stacking x and y
  # numpy is very particular about dimensions
  # here x is 2d but y 1d(vector), now to reshape y to 2d, we need call reshape
  # and also need to pass dimensions
  # here (-1,1) means make this 2d array where -1 means len(y)

  data = np.hstack((X,np.reshape(y,(-1,1))))

  return data, X, y

In [ ]:
print(len(train[train["class"]==1]))  # gamma
print(len(train[train["class"]==0]))

here we have 7k gamma and 4k hydra, this will become a issue, so we need to oversample our training data sets means to increase the number of values, so they match with each other.

do reslove this we can import RandomOverSampler

In [ ]:
def scale_dataset(dataframe, oversampler=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversampler:
    ros = RandomOverSampler()
    X,y = ros.fit_resample(X,y)   # says take less class and keep sampling from their to increase the dataset of that smaller class, so that they can match

  data = np.hstack((X,np.reshape(y,(-1,1))))

  return data, X, y

In [ ]:
train, X_train, y_train = scale_dataset(train, oversampler= True)